# CSV Comparison Report Generator

Cassidy Denault - Earth Economics - 2025

Compares two csv's: one with info gathered from analysts (expected, benchmark, ground truth) and one with info gathered from our AI powered application (generated) for the *identified important fields*

Uses pretrained BERT model w/ cosine similarity.

Creates a downloadable csv containing contents of both for easy comparison + similarity score evaluations

## User Guide

To conduct an analysis for 2 research papers. Open the analysis [data table](https://docs.google.com/spreadsheets/d/1jCMh0DtNmXQ8VLkwhWsx8AMREu6xhxVOgg50rKbGglQ/edit?usp=sharing) & follow the steps below.



1.   Run the **import dependencies** cells
2.   Generate AI model xlsx file. You'll need to **export the result table as a csv.** **Record the estimated run time in seconds.**
3.   Get expected file csv. Should be ONLY column name & value
4.   **Upload** both in files tab to the left
5.   Right click to get their file path and put in correct read_csv function
6.   To run program and get downloadable **select the Upload Data Cell**
7. Go to **Runtime tab above -> Run Cell And Below**.
8. Result is downloaded
9. Add the scores + other data to the analysis table [here](https://docs.google.com/spreadsheets/d/1jCMh0DtNmXQ8VLkwhWsx8AMREu6xhxVOgg50rKbGglQ/edit?usp=sharing)


##Import Dependencies

In [24]:
#from google.colab import drive # allow retrieval of files from your drive
from google.colab import files # for downloading result
import pandas as pd

# cosine similarity models
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
#drive.mount('/content/drive')

Mounted at /content/drive


## Upload Data

Upload data to the files tab (it will only exist there for local runtime). You can right click to get path and insert to the read_csv function.

In [6]:
# Upload TRAINING DATA CSV
expected = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/colab_data/new_expected.csv')

# Upload AI GENERATED CSV
derived = pd.read_csv('/content/Earth Economics Analysis - derived.csv')
derived

,Bibliography General,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 226,Unnamed: 227,Unnamed: 228,Regularized Value,Unnamed: 230,Unnamed: 231,Unnamed: 232,Unnamed: 233,Unnamed: 234,Unnamed: 235
0,Authors (semi-colon separated),Title,Year,Month,Day,Publication Type,Editors,Publisher,URL,DOI,...,Confidence Interval High,Probability,Other Probability,Low,High,Single,Dimension Type,Dimension Unit,Time Unit,Notes
1,NaN,Analysis Willingness to pay for unfamiliar pub...,2015,February,2,Journal,NaN,Elsevier B.V.,NaN,10.1016/j.ecolecon.2015.02.007,...,NaN,NaN,NaN,NaN,NaN,NaN,Areas,NaN,NaN,NaN


## Clean Data

In [7]:
derived


,Bibliography General,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 226,Unnamed: 227,Unnamed: 228,Regularized Value,Unnamed: 230,Unnamed: 231,Unnamed: 232,Unnamed: 233,Unnamed: 234,Unnamed: 235
0,Authors (semi-colon separated),Title,Year,Month,Day,Publication Type,Editors,Publisher,URL,DOI,...,Confidence Interval High,Probability,Other Probability,Low,High,Single,Dimension Type,Dimension Unit,Time Unit,Notes
1,NaN,Analysis Willingness to pay for unfamiliar pub...,2015,February,2,Journal,NaN,Elsevier B.V.,NaN,10.1016/j.ecolecon.2015.02.007,...,NaN,NaN,NaN,NaN,NaN,NaN,Areas,NaN,NaN,NaN


In [8]:
expected

,NLCD,Land Cover,Land Cover Specific,Land Cover Sub-Specific,Proximity,Within,Climate Group,Climate Type,Elevation,Water,...,Temporal Unit (Regularized),Low Value (Converted),High Value (Converted),Single Value (Converted),Spatial Unit (Converted),Temporal Unit (Converted),Currency Year (Converted),Comments,Link to Article,State
0,NaN,Underwater,Deep Seabed,Other,"Coastal Area, Shoreline",Coastal Area,E - Polar and Alpine,EF - Permanent Ice,NaN,Saline,...,year,660.056048,1863.344087,NaN,Acres,year,2023,NaN,NaN,accepted


### Function to Extract the Bibliographic Journal Info from Training data

And add the individual fields to the expected_dict

In [9]:
import re

def parse_reference(expected_dict):
    """
    Parse the 'Full Reference' entry in the expected dictionary to extract bibliographic details.

    Args:
        expected_dict: Dictionary containing a 'Full Reference' key

    Returns:
        Updated dictionary with extracted bibliographic details
    """
    if 'Full Reference' not in expected_dict:
        print("No 'Full Reference' key found in the dictionary")
        return expected_dict

    reference = expected_dict['Full Reference'][0]

    # Extract authors (everything before the year)
    year_match = re.search(r'(\d{4})', reference)
    if year_match:
        year_pos = year_match.start()
        authors = reference[:year_pos].strip()
        # Remove the trailing period if present
        if authors.endswith('.'):
            authors = authors[:-1]
        expected_dict['Authors'] = {0: authors}

        # Extract publication year
        year = year_match.group(1)
        expected_dict['Publication Year'] = {0: year}

        # Extract title (between year and journal title)
        # Find position after year and period
        after_year_pos = reference.find('.', year_pos) + 1

        # Find the journal pattern
        journal_match = re.search(r'\.([^\.]+?)\s+(\d+):', reference)

        if journal_match:
            journal_start_pos = journal_match.start() + 1  # +1 to skip the period
            title = reference[after_year_pos:journal_start_pos].strip()
            expected_dict['Title'] = {0: title}

            # Extract journal title
            journal_title = journal_match.group(1).strip()
            expected_dict['Journal Title'] = {0: journal_title}

            # Extract issue
            issue = journal_match.group(2)
            expected_dict['Journal Issue'] = {0: issue}

            # Extract page numbers
            pages_match = re.search(r'(\d+)-(\d+)', reference)
            if pages_match:
                start_page = pages_match.group(1)
                end_page = pages_match.group(2)
                expected_dict['Journal Start Page'] = {0: start_page}
                expected_dict['Journal End Page'] = {0: end_page}

    return expected_dict

# Test with the sample data
# sample_dict = {
#     'Full Reference': {0: 'Aanesen, M., Armstrong, C. W., Czajkowski, M., Falk-Petersen, J., Hanley, N., Navrud, S. 2015. Willingness to pay for unfamiliar public goods: Preserving cold-water coral in Norway. Ecological Economics 112: 53-67.'}
# }

# result = parse_reference(sample_dict)
# print("Updated dictionary:")
# for key, value in result.items():
#     print(f"{key}: {value}")

### Dictionary the CSVs

Expected dictionary (training data) in format:

*   Expected dictionary (training data) in format:

    ***{column name} : {value in a list}***

    Access: expected_dict['Author(s) (Primary)'][0]


*   Derived Dictionary (ai gen data) in format:

    ***{column name} : {value}***

    Access: derived_dict['Author(s) (Primary)']





In [10]:
expected_dict = expected.to_dict()


# Make the derived csv into a dictionary with row 0 contents as the key and row 1 contents as the value
derived_dict = derived.iloc[0:2].T.set_index(0).squeeze().to_dict()
#derived_dict

# print dictionary
# print(expected_dict['Author(s) (Primary)'][0])
# expected_dict


In [11]:
expected_dict = parse_reference(expected_dict) # parse the individual bibliographic cols

# print(derived_dict['Author(s) (Primary)'][0])

## Pre Processing

Import the model

In [12]:
# from math import nan

# Initializing the Sentence Transformer model using BERT with mean-tokens pooling
model = SentenceTransformer('bert-base-nli-mean-tokens')

# Load the model
# model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.77k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### Detail Columns to Compare

This dictionary details the columns that will be compared by the report generator.



We match the column name in training data csv & column name in ai generated csv. The training data has different column names than the desired ai generated template for the same field.



Additionally, only the fields identified by the sponsor as important (** +) are listed.

In [13]:
exp_to_derived = {
    # {column name in expected} : {column name in derived ai generated}
    "Authors": "Authors (semi-colon separated)",
    # "Full Reference": "",
    "Journal Title": "Journal Title",
    "Journal Issue": "Journal Issue",
    "Journal Start Page": "Journal Start Page",
    "Journal End Page": "Journal End Page",
    "Title": "Title",
    "Publication Year": "Year",
    "Reference Type": "Publication Type",
    "Continent": "Continent (semi-colon separated)",
    "Country": "Country (semi-colon separated)",
    #"": "Subcountry (semi-colon separated)", # could not find this in the training data cols
    #"": "Ecosystem General", # could not find this in the training data cols
    "Ecosystem Service Category": "Ecosystem Service Category",
    "Ecosystem Service General": "Ecosystem Service General",
    "Valuation Methodology General": "Valuation Methodology General",
    "Valuation Methodology Specific": "Valuation Methodology Specific",

    # the fields below not marked as important, but added for thoroughness
    "Valuation Type": "Calculation Type"
    # "": "",

}

### Report Generator Code

Function to compare cell & generate a similarity score

Input:
*   expected accepted dictionary
*   ai derived dictionary
*   column name matching dictionary
*   model you wish to use to generate similarity scores


Returns:
*   report containing contents and scores in form of a data frame


In [18]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def generate_comparison_csv(expected_dict, derived_dict, exp_to_derived, model):
    """
    Generate a CSV comparing values between expected_dict and derived_dict based on mapping in exp_to_derived.

    Args:
        expected_dict: Dictionary with expected values
        derived_dict: Dictionary with derived values
        exp_to_derived: Mapping from expected_dict keys to derived_dict keys
        model: Sentence transformer model for calculating similarity

    Returns:
        DataFrame containing the comparison
    """
    # Initialize lists for rows
    expected_values = []
    derived_values = []
    similarity_scores = []

    # Track sum for analysis
    sum_scores = 0
    valid_comparisons = 0

    # Create column list
    columns = list(exp_to_derived.keys())

    # Process each column
    for exp_col in columns:
        # Get corresponding derived column
        derived_col = exp_to_derived[exp_col]

        # Get values (handle missing keys gracefully)
        expected_value = str(expected_dict.get(exp_col, {}).get(0, "nan"))
        derived_value = str(derived_dict.get(derived_col, "nan"))

        # Clean up values
        expected_value = expected_value if expected_value != "nan" else ""
        derived_value = derived_value if derived_value != "nan" else ""

        # Add to values lists
        expected_values.append(expected_value)
        derived_values.append(derived_value)

        # Calculate similarity
        if expected_value == "" and derived_value == "":
            # Both empty
            similarity_scores.append(1.0)
            sum_scores += 1.0
        elif expected_value == "" and derived_value != "":
            # No expected, derived generated something
            similarity_scores.append(1.0)
            sum_scores += 1.0
        elif expected_value != "" and derived_value == "":
            # Expected something, derived generated nothing
            similarity_scores.append(0.0)
            valid_comparisons += 1
        else:
            # Both have values, calculate similarity
            try:
                expected_embeddings = model.encode(expected_value)
                derived_embeddings = model.encode(derived_value)

                score = cosine_similarity([expected_embeddings], [derived_embeddings], dense_output=True)[0][0]

                similarity_scores.append(score)
                sum_scores += score
                valid_comparisons += 1

            except Exception as e:
                print(f"Error calculating similarity for {exp_col}: {e}")
                similarity_scores.append(0.0)

    # Calculate average similarity score
    avg_score = sum_scores / len(columns) if columns else 0

    # Calculate average similarity score without both empty and generating when not expected
    avg_score_sub = (sum_scores - (len(columns) - valid_comparisons)) / valid_comparisons if valid_comparisons > 0 else 0

    # Create the DataFrame
    df = pd.DataFrame({
        'Column': columns,
        'Expected': expected_values,
        'Derived': derived_values,
        'Similarity': similarity_scores
    })

    # Add a row for average similarity
    avg_row = pd.DataFrame({
        'Column': ['Average Similarity'],
        'Expected': [''],
        'Derived': [''],
        'Similarity': [avg_score]
    })

    # Add a row for subset average similarity
    avg_row_sub = pd.DataFrame({
        'Column': ['Average Similarity of Filled Expected Fields'],
        'Expected': [''],
        'Derived': [''],
        'Similarity': [avg_score_sub]
    })

    result_df = pd.concat([df, avg_row, avg_row_sub], ignore_index=True)
    #result_df = pd.concat([df, avg_row_sub], ignore_index=True)

    return result_df

# Example usage:

# Sample dictionaries
# expected_dict = {
#     'Authors': {0: 'Aanesen, M., Armstrong, C. W., Czajkowski, M., Falk-Petersen, J., Hanley, N., Navrud, S.'},
#     'Journal Title': {0: 'Ecological Economics'},
#     'Journal Issue': {0: '112'},
#     'Journal Start Page': {0: '53'},
#     'Journal End Page': {0: '67'},
#     'Title': {0: 'Willingness to pay for unfamiliar public goods: Preserving cold-water coral in Norway'},
#     'Publication Year': {0: '2015'},
#     'Reference Type': {0: 'Journal Article'},
#     'Continent': {0: 'Europe'},
#     'Country': {0: 'Norway'},
#     'Ecosystem Service Category': {0: 'Cultural'},
#     'Ecosystem Service General': {0: 'Recreation'},
#     'Valuation Methodology General': {0: 'Stated Preference'},
#     'Valuation Methodology Specific': {0: 'Choice Experiment'},
#     'Valuation Type': {0: 'Primary'}
# }

# derived_dict = {
#     'Authors (semi-colon separated)': 'Aanesen, M.; Armstrong, C. W.; Czajkowski, M.; Falk-Petersen, J.; Hanley, N.; Navrud, S.',
#     'Journal Title': 'Ecological Economics',
#     'Journal Issue': '112',
#     'Journal Start Page': '53',
#     'Journal End Page': '67',
#     'Title': 'Willingness to pay for unfamiliar public goods: Preserving cold-water coral in Norway',
#     'Year': '2015',
#     'Publication Type': 'Journal Article',
#     'Continent (semi-colon separated)': 'Europe',
#     'Country (semi-colon separated)': 'Norway',
#     'Ecosystem Service Category': 'Provisioning',
#     'Ecosystem Service General': 'Recreation',
#     'Valuation Methodology General': 'Stated Preference',
#     'Valuation Methodology Specific': 'Choice Experiment',
#     'Calculation Type': 'Primary'
# }

# exp_to_derived = {
#     "Authors": "Authors (semi-colon separated)",
#     "Journal Title": "Journal Title",
#     "Journal Issue": "Journal Issue",
#     "Journal Start Page": "Journal Start Page",
#     "Journal End Page": "Journal End Page",
#     "Title": "Title",
#     "Publication Year": "Year",
#     "Reference Type": "Publication Type",
#     "Continent": "Continent (semi-colon separated)",
#     "Country": "Country (semi-colon separated)",
#     "Ecosystem Service Category": "Ecosystem Service Category",
#     "Ecosystem Service General": "Ecosystem Service General",
#     "Valuation Methodology General": "Valuation Methodology General",
#     "Valuation Methodology Specific": "Valuation Methodology Specific",
#     "Valuation Type": "Calculation Type"
# }





## Data Analysis

In [19]:
# Generate the comparison
result = generate_comparison_csv(expected_dict, derived_dict, exp_to_derived, model)


## Download Report

In [23]:
# Save to CSV

from datetime import datetime

# Generate the filename with the current date
now = datetime.now()
filename = f"comparison_report_{now.month}_{now.day}_{now.year}.csv"
    # example: comparison_results_5_13_2025.csv for date 5/13/2025

# Save the DataFrame to the generated filename
result.to_csv(filename, index=False)

# Print the filename
#print(f"Comparison results saved to: {filename}")


Comparison results saved to: comparison_results_5_13_2025.csv


In [21]:
files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Sources & Research


*   [Used this techinque heavily](https://medium.com/@ahmedmellit/text-similarity-implementation-using-bert-embedding-in-python-1efdb5194e65)
*   [Text similarity basics, cosine especially](https://www.newscatcherapi.com/blog/ultimate-guide-to-text-similarity-with-python#toc-4)
*   cosine similarity via sklearn



